In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from env import user, pwd, host
import os
from pyspark.sql.functions import month, year, quarter

In [2]:
pandas_df = pd.read_csv('311.csv')

In [3]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 10:16:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.createDataFrame(pandas_df)

In [5]:
df.show(5)

23/05/22 10:16:14 WARN TaskSetManager: Stage 0 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO| -2.012604167|        YES|     Storm Water|Removal Of Obstru...|4.322222222|     Closed| svcCRMSS|2215  GOL

In [6]:
pandas_df2 = pd.read_csv('dept.csv')

In [7]:
df2 = spark.createDataFrame(pandas_df2)

In [8]:
df2.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [9]:
df.write.json('data/source1_json',mode='overwrite')

23/05/22 10:16:16 WARN TaskSetManager: Stage 3 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [10]:
(
df.write.format('csv')
    .mode('overwrite')
    .option('header',True)
    .save('data/source1_csv')

)

23/05/22 10:16:19 WARN TaskSetManager: Stage 4 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [11]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: long (nullable = true)



In [12]:
df = df.withColumnRenamed('SLA_due_date','case_due_date')

In [13]:
df = df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

In [14]:
df.select('case_closed','case_late').show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



23/05/22 10:16:20 WARN TaskSetManager: Stage 5 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [15]:
df = df.withColumn("council_district", col("council_district").cast("string"))

In [16]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- case_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [17]:
fmt = "M/d/yy H:mm"
df = (
    df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

In [18]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [19]:
df = df.withColumn("request_address", trim(lower(df.request_address)))

In [51]:
df = df.withColumn(
    "num_weeks_late", F.round(expr("num_days_late / 7 AS num_weeks_late"),1)
)

In [21]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- num_weeks_late: double (nullable = true)



In [22]:
df = df.withColumn("council_district", col("council_district").cast("int"))

In [23]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)
 |-- num_weeks_late: double (nullable = true)



In [24]:
df = df.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)

In [25]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)



In [26]:
df = df.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))

In [27]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)



In [28]:
df = (
    df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [29]:
df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

23/05/22 10:16:21 WARN TaskSetManager: Stage 6 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|       true|2018-01-01 00:42:00|2018-01-01 00:42:00|    1967|             0|            0|
|       true|2018-01-01 00:46:00|2018-01-01 00:46:00|    1967|             0|            0|
|       true|2018-01-01 00:48:00|2018-01-01 00:48:00|    1967|             0|            0|
|       true|2018-01-01 01:29:00|2018-01-01 01:29:00|    1967|             0|            0|
|       true|2018-01-01 01:34:00|2018-01-01 01:34:00|    1967|             0|            0|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows



In [30]:
df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|      false|2018-01-02 09:39:00|2018-01-02 09:39:00|    1966|             0|         1966|
|      false|2018-01-02 10:49:00|2018-01-02 10:49:00|    1966|             0|         1966|
|      false|2018-01-02 13:45:00|2018-01-02 13:45:00|    1966|             0|         1966|
|      false|2018-01-02 14:09:00|2018-01-02 14:09:00|    1966|             0|         1966|
|      false|2018-01-02 14:34:00|2018-01-02 14:34:00|    1966|             0|         1966|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows



23/05/22 10:16:21 WARN TaskSetManager: Stage 7 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


## 1 how old is currently open issue

In [31]:
df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).sort(df.case_age.desc()).show(5)

23/05/22 10:16:22 WARN TaskSetManager: Stage 8 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|      false|2018-01-02 09:39:00|2018-01-02 09:39:00|    1966|             0|         1966|
|      false|2018-01-02 10:49:00|2018-01-02 10:49:00|    1966|             0|         1966|
|      false|2018-01-02 13:45:00|2018-01-02 13:45:00|    1966|             0|         1966|
|      false|2018-01-02 14:09:00|2018-01-02 14:09:00|    1966|             0|         1966|
|      false|2018-01-02 14:34:00|2018-01-02 14:34:00|    1966|             0|         1966|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows



## 2 how many Stray Animal cases

In [32]:
df.groupby(df.service_request_type).agg(F.count(df.service_request_type)).show()

23/05/22 10:16:23 WARN TaskSetManager: Stage 9 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------------------------+
|service_request_type|count(service_request_type)|
+--------------------+---------------------------+
|Minimum Housing-O...|                        897|
|        Tree Removal|                         15|
| Service Information|                         13|
|    Sign Maintenance|                          7|
|Vacant Lot-Privat...|                        687|
|Graffiti: Private...|                       1065|
|Traffic Sign Graf...|                        397|
|License Renewal I...|                        128|
|   Sewer Line Broken|                        170|
|Zoning: Multi-Fam...|                         86|
|Engineering Inves...|                         53|
|    Zoning: Setbacks|                         85|
|  Traffic Sign Faded|                        291|
|     Permits, Fences|                        138|
|Certificates of O...|                        507|
|  Dead Animal - Misc|                       1396|
|Garage Sales No P...|         

In [33]:
df.groupby(df.service_request_type).agg(F.count(df.service_request_type)).where(
    F.col('service_request_type') == 'Stray Animal').show()

23/05/22 10:16:25 WARN TaskSetManager: Stage 12 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------------------------+
|service_request_type|count(service_request_type)|
+--------------------+---------------------------+
|        Stray Animal|                       3497|
+--------------------+---------------------------+



## 3 how many Field Ops cases not classified as Officer Standby

In [34]:
df.groupby(df.dept_division,df.service_request_type).agg(F.count(df.dept_division)).where(
    F.col('dept_division') == 'Field Operations').show()

23/05/22 10:16:26 WARN TaskSetManager: Stage 15 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+----------------+--------------------+--------------------+
|   dept_division|service_request_type|count(dept_division)|
+----------------+--------------------+--------------------+
|Field Operations|Aggressive Animal...|                3541|
|Field Operations|City Council Anim...|                  46|
|Field Operations|Animal Cruelty(Cr...|                 479|
|Field Operations|Public Nuisance(O...|                1531|
|Field Operations|        Stray Animal|                3497|
|Field Operations|Aggressive Animal...|                 746|
|Field Operations|Injured Animal(Cr...|                1021|
|Field Operations|     Officer Standby|                 384|
|Field Operations|Animal Bite(Criti...|                  82|
|Field Operations|Trapped/Confined ...|                1299|
|Field Operations|Animal Bite(Non-C...|                 544|
|Field Operations|      Animal Neglect|                1928|
|Field Operations|Animal Permits Re...|                 380|
+----------------+------

## 4 council_district converted to string

In [35]:
# df = df.withColumn("council_district", col("council_district").cast("string"))

## 5 extract year from case_closed_date

In [41]:
df.withColumn('year_closed', F.year(F.col('case_closed_date'))).groupby(
    'year_closed').agg(F.count(df.case_closed_date)).show()

23/05/22 10:26:10 WARN TaskSetManager: Stage 23 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-----------------------+
|year_closed|count(case_closed_date)|
+-----------+-----------------------+
|       2018|                 100000|
+-----------+-----------------------+



In [42]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)



## 6 convert num_days_late from days to hours: num_hours_late

In [48]:
df = df.withColumn(
    "num_hours_late", F.round(expr("num_days_late * 24 AS num_hours_late"),1)
)

In [49]:
df.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- num_hours_late: double (nullable = true)



## 7 join

In [52]:
df.join(df2, how='left', on=df.dept_division == df2.dept_division).show(1)

23/05/22 10:59:32 WARN TaskSetManager: Stage 50 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------------------+-------------------+-------------------+---------+-------------+-----------+----------------+--------------------+--------+-----------+---------+--------------------+----------------+--------------+-------+--------+--------------+-------------+--------------+----------------+--------------------+----------------------+-------------------+
|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|SLA_days|case_status|source_id|     request_address|council_district|num_weeks_late|zipcode|case_age|days_to_closed|case_lifetime|num_hours_late|   dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------+-------------------+-------------------+-------------------+---------+-------------+-----------+----------------+--------------------+--------+-----------+---------+--------------------+----------------+--------------+-------+--------+-------

## 8 any cases without a request source

In [57]:
df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).filter(F.col('source_id').isNull()).show(5)

23/05/22 11:11:16 WARN TaskSetManager: Stage 59 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.
23/05/22 11:11:16 WARN TaskSetManager: Stage 60 contains a task of very large size (1356 KiB). The maximum recommended task size is 1000 KiB.


+-----------+----------------+----------------+--------+--------------+-------------+
|case_closed|case_opened_date|case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+----------------+----------------+--------+--------------+-------------+
+-----------+----------------+----------------+--------+--------------+-------------+



23/05/22 11:11:17 WARN TaskSetManager: Stage 61 contains a task of very large size (1350 KiB). The maximum recommended task size is 1000 KiB.


In [60]:
df.describe().show()

23/05/22 11:18:15 WARN TaskSetManager: Stage 65 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------------+-------------+----------------+--------------------+--------+-----------+------------------+--------------------+------------------+--------------+------------------+------------------+--------------+------------------+--------------+
|summary|           case_id|num_days_late|   dept_division|service_request_type|SLA_days|case_status|         source_id|     request_address|  council_district|num_weeks_late|           zipcode|          case_age|days_to_closed|     case_lifetime|num_hours_late|
+-------+------------------+-------------+----------------+--------------------+--------+-----------+------------------+--------------------+------------------+--------------+------------------+------------------+--------------+------------------+--------------+
|  count|            100000|       100000|          100000|              100000|  100000|     100000|            100000|              100000|            100000|        100000|            100000|            10000

## 9 Top 10 service requests types by count

In [67]:
df.groupby(df.service_request_type).agg(F.count(df.service_request_type)).sort(
    F.count(df.service_request_type).desc()).show(10)

23/05/22 11:22:44 WARN TaskSetManager: Stage 83 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------------------------+
|service_request_type|count(service_request_type)|
+--------------------+---------------------------+
|           No Pickup|                       8932|
|Overgrown Yard/Trash|                       6853|
|Front Or Side Yar...|                       5338|
|        Bandit Signs|                       4781|
|Aggressive Animal...|                       3541|
|        Stray Animal|                       3497|
|        Damaged Cart|                       3465|
|Cart Exchange Req...|                       3282|
|Right Of Way/Side...|                       3121|
|Junk Vehicle On P...|                       2897|
+--------------------+---------------------------+
only showing top 10 rows



## 10 Top 10 service requests types by average days_late

In [75]:
df.groupby(df.service_request_type).agg(F.round(F.avg(df.num_days_late),1)).sort(
    F.avg(df.num_days_late).desc()).show(11)

23/05/22 11:49:01 WARN TaskSetManager: Stage 104 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------------------------+
|service_request_type|round(avg(num_days_late), 1)|
+--------------------+----------------------------+
|Request for Resea...|                         NaN|
|  Zoning: Junk Yards|                       178.3|
|Emergency: Main a...|                       173.5|
|Status: On-Going ...|                       159.9|
|Zoning: Recycle Yard|                       152.2|
|           Complaint|                       136.8|
|Markings Installa...|                       111.4|
|             Vendors|                        71.2|
|Donation Containe...|                        61.6|
|   No Address Posted|                        61.1|
|         Curbstoning|                        54.4|
+--------------------+----------------------------+
only showing top 11 rows



## 11 days late functionally dependent upon department

In [79]:
df.groupby(df.dept_division).agg(F.round(F.avg(df.num_days_late),1)).sort(
    F.avg(df.num_days_late).desc()).show(40)

23/05/22 11:53:27 WARN TaskSetManager: Stage 116 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------------------------+
|       dept_division|round(avg(num_days_late), 1)|
+--------------------+----------------------------+
|          District 2|                         NaN|
|          District 1|                         NaN|
|Director's Office...|                       159.9|
|Code Enforcement ...|                       152.2|
|     311 Call Center|                       112.0|
|Engineering Division|                        30.2|
|              Trades|                        10.3|
|         Solid Waste|                         2.6|
|Clean and Green N...|                         2.4|
|       Miscellaneous|                        -1.7|
|           Tree Crew|                        -1.7|
|    Facility License|                        -1.8|
|    Waste Collection|                        -1.8|
|              Vector|                        -2.2|
|               Shops|                        -3.0|
|     Clean and Green|                        -3.2|
|           

## 12 days late functionally dependent upon department and request type

In [85]:
df.groupby(df.dept_division,df.service_request_type).agg(F.round(F.avg(df.num_days_late),1)).sort(
    F.avg(df.num_days_late).desc()).show(400)

23/05/22 11:56:24 WARN TaskSetManager: Stage 134 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+----------------------------+
|       dept_division|service_request_type|round(avg(num_days_late), 1)|
+--------------------+--------------------+----------------------------+
|          District 2|Request for Resea...|                         NaN|
|          District 1|Request for Resea...|                         NaN|
|    Code Enforcement|  Zoning: Junk Yards|                       178.3|
|   Dangerous Premise|Emergency: Main a...|                       173.5|
|Director's Office...|Status: On-Going ...|                       159.9|
|Code Enforcement ...|Zoning: Recycle Yard|                       152.2|
|     311 Call Center|           Complaint|                       136.8|
|  Signs and Markings|Markings Installa...|                       111.4|
|    Code Enforcement|             Vendors|                        71.2|
|    Code Enforcement|Donation Containe...|                        61.6|
|    Code Enforcement|   No Address Posted|        